In [ ]:
# | default_exp agents.planner

In [9]:
# | export
from enum import Enum
from typing import List, Dict, Any, Optional, Tuple
from pydantic import BaseModel, Field, ValidationError
import json
import re
from rich.console import Console
import time

from agentic.core.agent import Agent, AgentConfig
from agentic.llms.client import LLMClient

# Handle optional dependencies
try:
    import json5
    HAS_JSON5 = True
except ImportError:
    HAS_JSON5 = False

try:
    from jsonschema import validate, ValidationError as JsonSchemaValidationError
    HAS_JSONSCHEMA = True
except ImportError:
    HAS_JSONSCHEMA = False

class ExecutionMode(Enum):
    SEQUENTIAL = "sequential"
    PARALLEL = "parallel"

class Resources(BaseModel):
    previous: List[str] = Field(default_factory=list, description="Resources created in previous steps")
    required: List[str] = Field(default_factory=list, description="Resources helpful or required for this/current next step")

class ActionStep(BaseModel):
    step: int = Field(..., description="The step number")
    action: str = Field(..., description="The action description")
    tool: Optional[str] = Field(None, description="The tool to use (from available tools)")
    purpose: str = Field(..., description="The purpose of this step")
    sub_steps: List[str] = Field(default_factory=list, description="Sub-steps if any")
    introspect_after: bool = Field(True, description="Whether to call introspect after this step")
    system_prompt: str = Field(..., description="Self-sufficient system prompt for this subtask")
    user_prompt: str = Field(..., description="Self-sufficient user prompt for this subtask")
    introspect_prompt: str = Field(..., description="Template prompt for introspect")
    resources: Resources = Field(default_factory=Resources, description="Resources for this subtask")
    execution_mode: ExecutionMode = Field(ExecutionMode.SEQUENTIAL, description="Sequential or parallel execution")

class Task(BaseModel):
    id: str = Field(..., description="Unique task ID, e.g., T001")
    name: str = Field(..., description="Concise task name")
    description: str = Field(..., description="Detailed task description")
    dependencies: List[str] = Field(default_factory=list, description="List of dependent task IDs")
    tools: List[str] = Field(default_factory=list, description="List of tool names")
    actions: List[ActionStep] = Field(default_factory=list, description="List of action steps")
    success_criteria: str = Field(..., description="Criteria for task success")
    expected_outputs: List[str] = Field(default_factory=list, description="Expected output artifacts")
    resources: Resources = Field(default_factory=Resources, description="Overall resources for the task")

class ExecutionPlan(BaseModel):
    sequential_phases: List[List[str]] = Field(default_factory=list, description="Phases of task IDs")
    parallel_groups: List[List[str]] = Field(default_factory=list, description="Parallel task groups")
    critical_path: List[str] = Field(default_factory=list, description="Critical path task IDs")
    total_actions: int = Field(0, description="Total number of actions")
    ordering: List[str] = Field(default_factory=list, description="Overall task ordering")

class AnalysisResult(BaseModel):
    tasks: List[Task] = Field(default_factory=list, description="List of decomposed tasks")
    execution_plan: ExecutionPlan = Field(..., description="Execution plan")
    tools_needed: List[str] = Field(default_factory=list, description="Unique tools needed")
    success_criteria: List[str] = Field(default_factory=list, description="Overall success criteria")
    resources: Resources = Field(default_factory=Resources, description="Overall resources")

class JsonSchemaRepairAgent(Agent):
    """Agent for static checks and LLM-based repair of JSON or schema issues"""

    def __init__(self, config: AgentConfig):
        super().__init__(config)
        self.console = Console()

    def repair_json(self, raw_response: str, user_input: str) -> List[Dict[str, Any]]:
        """Repair JSON with static checks, library-based fixes, and LLM fallback"""
        self.console.print("[DEBUG] Starting JSON repair process")

        # Step 1: Static checks and cleaning
        cleaned, static_errors = self._static_check_and_clean(raw_response)
        self.console.print(f"[DEBUG] Static errors: {static_errors}")

        # Step 2: Try parsing JSON
        task_data = []
        if not static_errors:
            try:
                task_data = json.loads(cleaned)
                if not isinstance(task_data, list):
                    static_errors.append("JSON is not a list")
                    task_data = self._fix_non_list_json(cleaned)
                self.console.print(f"[DEBUG] Parsed {len(task_data)} tasks")
                return task_data  # Return if JSON is valid
            except json.JSONDecodeError as je:
                static_errors.append(f"JSON parsing failed: {str(je)}")
                task_data = []

        # Step 3: Validate tasks only if static checks failed
        schema_errors = []
        valid_tasks = task_data
        if static_errors and task_data:
            if HAS_JSONSCHEMA:
                valid_tasks, schema_errors = self._validate_with_jsonschema(task_data)
            else:
                valid_tasks, schema_errors = self._validate_with_pydantic(task_data)
            self.console.print(f"[DEBUG] Schema errors: {schema_errors}")
            self.console.print(f"[DEBUG] Valid tasks before repair: {len(valid_tasks)}")

        # Step 4: Use LLM repair if any errors
        if static_errors or schema_errors:
            self.console.print("[DEBUG] Invoking LLM for repair")
            repaired_tasks = self._llm_repair(raw_response, user_input, static_errors, schema_errors, valid_tasks)
            valid_tasks = repaired_tasks if repaired_tasks else valid_tasks

        # Step 5: Final validation with attempt to fix
        final_tasks = []
        for idx, task in enumerate(valid_tasks):
            try:
                Task.model_validate(task)
                final_tasks.append(task)
            except ValidationError as ve:
                self.console.print(f"⚠️ Task {idx+1} invalid: {ve}")
                fixed_task = self._fix_task_schema_issues(task, idx + 1)
                try:
                    Task.model_validate(fixed_task)
                    final_tasks.append(fixed_task)
                except ValidationError as ve2:
                    self.console.print(f"⚠️ Task {idx+1} unrepairable: {ve2}")
                    continue

        self.console.print(f"[DEBUG] Final repaired tasks: {len(final_tasks)}")
        return final_tasks if final_tasks else []

    def _static_check_and_clean(self, raw_response: str) -> Tuple[str, List[str]]:
        """Perform static checks and clean JSON"""
        errors = []
        response = raw_response.strip()

        # Handle code blocks
        if "```json" in response:
            start = response.find("```json") + 7
            end = response.rfind("```")
            if start > 6 and end > start:
                response = response[start:end].strip()
            else:
                errors.append("Invalid code block boundaries")
                return response, errors

        # Check for JSON boundaries
        if not (response.startswith("[") and response.endswith("]")) and not (response.startswith("{") and response.endswith("}")):
            errors.append("No valid JSON boundaries found")
            return response, errors

        # Check bracket balance
        bracket_count = 0
        square_count = 0
        in_string = False
        for char in response:
            if char == '"' and (response[max(0, response.index(char)-1)] != "\\" if response.index(char) > 0 else True):
                in_string = not in_string
            if not in_string:
                if char == "{": bracket_count += 1
                elif char == "}": bracket_count -= 1
                elif char == "[": square_count += 1
                elif char == "]": square_count -= 1
            if bracket_count < 0 or square_count < 0:
                errors.append("Unbalanced brackets")
                break

        # Try library-based parsing
        if HAS_JSON5:
            try:
                parsed = json5.loads(response)
                return json.dumps(parsed), errors
            except Exception as e:
                errors.append(f"json5 parsing failed: {str(e)}")

        # Try standard JSON
        try:
            json.loads(response)
            return response, errors
        except json.JSONDecodeError as e:
            errors.append(f"JSON parsing failed: {str(e)}")
            return self._extract_json_manually(response), errors

    def _extract_json_manually(self, response: str) -> str:
        """Manual JSON extraction as last resort"""
        bracket_start = response.find("[")
        bracket_end = response.rfind("]")
        if bracket_start != -1 and bracket_end != -1 and bracket_start < bracket_end:
            potential_json = response[bracket_start:bracket_end + 1]
            try:
                json.loads(potential_json)
                return potential_json
            except json.JSONDecodeError:
                pass

        brace_start = response.find("{")
        brace_end = response.rfind("}")
        if brace_start != -1 and brace_end != -1 and brace_start < brace_end:
            potential_json = response[brace_start:brace_end + 1]
            try:
                parsed = json.loads(potential_json)
                return json.dumps([parsed] if isinstance(parsed, dict) else parsed)
            except json.JSONDecodeError:
                pass

        return "[]"

    def _validate_with_jsonschema(self, task_data: List[Dict[str, Any]]) -> Tuple[List[Dict[str, Any]], List[str]]:
        """Validate using jsonschema library"""
        task_schema = {
            "type": "array",
            "items": {
                "type": "object",
                "required": ["id", "name", "description", "dependencies", "tools", "actions", "success_criteria", "expected_outputs", "resources"],
                "properties": {
                    "id": {"type": "string", "pattern": "^T\\d+$"},
                    "name": {"type": "string", "minLength": 1},
                    "description": {"type": "string", "minLength": 1},
                    "dependencies": {"type": "array", "items": {"type": "string"}},
                    "tools": {
                        "type": "array",
                        "items": {
                            "type": "string",
                            "enum": ["fs_read", "fs_write", "execute_bash", "code_interpreter", "introspect", "debate_agent"]
                        }
                    },
                    "actions": {
                        "type": "array",
                        "items": {
                            "type": "object",
                            "required": ["step", "action", "purpose", "sub_steps", "introspect_after", "system_prompt", "user_prompt", "introspect_prompt", "resources", "execution_mode"],
                            "properties": {
                                "step": {"type": "integer", "minimum": 1},
                                "action": {"type": "string", "minLength": 1},
                                "tool": {"type": ["string", "null"]},
                                "purpose": {"type": "string", "minLength": 1},
                                "sub_steps": {"type": "array", "items": {"type": "string"}},
                                "introspect_after": {"type": "boolean"},
                                "system_prompt": {"type": "string", "minLength": 1},
                                "user_prompt": {"type": "string", "minLength": 1},
                                "introspect_prompt": {"type": "string", "minLength": 1},
                                "resources": {
                                    "type": "object",
                                    "required": ["previous", "required"],
                                    "properties": {
                                        "previous": {"type": "array", "items": {"type": "string"}},
                                        "required": {"type": "array", "items": {"type": "string"}}
                                    }
                                },
                                "execution_mode": {"type": "string", "enum": ["sequential", "parallel"]}
                            }
                        }
                    },
                    "success_criteria": {"type": "string", "minLength": 1},
                    "expected_outputs": {"type": "array", "items": {"type": "string"}},
                    "resources": {
                        "type": "object",
                        "required": ["previous", "required"],
                        "properties": {
                            "previous": {"type": "array", "items": {"type": "string"}},
                            "required": {"type": "array", "items": {"type": "string"}}
                        }
                    }
                }
            }
        }

        errors = []
        valid_tasks = []
        try:
            validate(task_data, task_schema)
            return task_data, errors
        except JsonSchemaValidationError as e:
            errors.append(f"Schema validation failed: {str(e)}")

        for idx, task in enumerate(task_data):
            try:
                validate(task, task_schema["items"])
                valid_tasks.append(task)
            except JsonSchemaValidationError as e:
                errors.append(f"Task {idx+1} validation failed: {str(e)}")
                fixed_task = self._fix_task_schema_issues(task, idx + 1)
                try:
                    validate(fixed_task, task_schema["items"])
                    valid_tasks.append(fixed_task)
                except JsonSchemaValidationError:
                    errors.append(f"Task {idx+1} unrepairable")
                    continue

        return valid_tasks, errors

    def _validate_with_pydantic(self, task_data: List[Dict[str, Any]]) -> Tuple[List[Dict[str, Any]], List[str]]:
        """Fallback validation using pydantic"""
        errors = []
        valid_tasks = []
        
        for idx, task in enumerate(task_data):
            try:
                Task.model_validate(task)
                valid_tasks.append(task)
            except ValidationError as ve:
                errors.append(f"Task {idx+1}: {str(ve)}")
                fixed_task = self._fix_task_schema_issues(task, idx + 1)
                try:
                    Task.model_validate(fixed_task)
                    valid_tasks.append(fixed_task)
                except ValidationError:
                    errors.append(f"Task {idx+1} unrepairable")
                    continue
        
        return valid_tasks, errors

    def _fix_task_schema_issues(self, task: Dict[str, Any], task_idx: int) -> Dict[str, Any]:
        """Fix common schema issues in tasks"""
        fixed_task = task.copy()
        
        # Ensure required fields
        defaults = {
            "id": f"T{task_idx:03d}",
            "name": fixed_task.get("name", "Unnamed Task"),
            "description": fixed_task.get("description", "Task description"),
            "dependencies": fixed_task.get("dependencies", []),
            "tools": fixed_task.get("tools", []),
            "actions": fixed_task.get("actions", []),
            "success_criteria": fixed_task.get("success_criteria", "Task completed successfully"),
            "expected_outputs": fixed_task.get("expected_outputs", []),
            "resources": fixed_task.get("resources", {"previous": [], "required": []})
        }
        for field, default_value in defaults.items():
            if field not in fixed_task:
                fixed_task[field] = default_value

        # Fix resources
        if not isinstance(fixed_task["resources"], dict):
            fixed_task["resources"] = {"previous": [], "required": []}
        for res_field in ["previous", "required"]:
            if res_field not in fixed_task["resources"]:
                fixed_task["resources"][res_field] = []

        # Fix actions
        if isinstance(fixed_task["actions"], list):
            for idx, action in enumerate(fixed_task["actions"]):
                if not isinstance(action, dict):
                    fixed_task["actions"][idx] = {}
                    action = fixed_task["actions"][idx]
                action_defaults = {
                    "step": idx + 1,
                    "action": action.get("action", "Perform task step"),
                    "tool": action.get("tool", None),
                    "purpose": action.get("purpose", "Execute task step"),
                    "sub_steps": action.get("sub_steps", []),
                    "introspect_after": action.get("introspect_after", True),
                    "system_prompt": action.get("system_prompt", "Technical agent for software tasks"),
                    "user_prompt": action.get("user_prompt", "Execute task step"),
                    "introspect_prompt": action.get("introspect_prompt", "Evaluate {performed_action}"),
                    "resources": action.get("resources", {"previous": [], "required": []}),
                    "execution_mode": action.get("execution_mode", "sequential")
                }
                for field, default_value in action_defaults.items():
                    if field not in action:
                        action[field] = default_value
                if not isinstance(action["resources"], dict):
                    action["resources"] = {"previous": [], "required": []}
                for res_field in ["previous", "required"]:
                    if res_field not in action["resources"]:
                        action["resources"][res_field] = []
                if action["execution_mode"] not in ["sequential", "parallel"]:
                    action["execution_mode"] = "sequential"

        return fixed_task

    def _fix_non_list_json(self, cleaned: str) -> List[Dict[str, Any]]:
        """Fix JSON that isn't a list"""
        try:
            data = json.loads(cleaned)
            if isinstance(data, dict):
                return [data]
            return []
        except json.JSONDecodeError:
            return []

    def _llm_repair(self, raw_response: str, user_input: str, static_errors: List[str], schema_errors: List[str], tasks: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
        """Use LLM to repair JSON and schema issues"""
        prompt = f"""
The following JSON response from a task decomposition is malformed or does not comply with the expected schema:
```
{raw_response}
```

Static check errors:
{json.dumps(static_errors, indent=2)}

Schema validation errors:
{json.dumps(schema_errors, indent=2)}

Original user input: '{user_input}'

Current tasks (may include invalid ones):
{json.dumps(tasks, indent=2)}

Repair the JSON to produce a valid list of tasks matching the schema:
- id: string (e.g., T001)
- name: string
- description: string
- dependencies: list of strings
- tools: list of strings (from {', '.join(TaskAnalyzer.AVAILABLE_TOOLS)})
- actions: list of objects with step (int), action (str), tool (str or null), purpose (str), sub_steps (list str), introspect_after (bool), system_prompt (str), user_prompt (str), introspect_prompt (str), resources (object with previous/required lists), execution_mode ("sequential" or "parallel")
- success_criteria: string
- expected_outputs: list of strings
- resources: object with previous/required lists

Rules:
- Preserve all information from the original response.
- Fix structural issues (e.g., missing brackets, invalid JSON).
- Correct schema issues (e.g., invalid execution_mode, missing fields) with sensible defaults or inference from user_input.
- Ensure all tasks are valid and include all required fields.
- Do not discard any tasks unless absolutely unrecoverable.
- Output a valid JSON array of tasks.
"""
        response = self._call_llm_with_retry(prompt)
        try:
            cleaned = self._clean_json(response)
            parsed = json.loads(cleaned)
            return parsed if isinstance(parsed, list) else [parsed]
        except json.JSONDecodeError as je:
            self.console.print(f"⚠️ LLM repair failed: {je}")
            return tasks  # Return original tasks if repair fails

    def _call_llm_with_retry(self, prompt: str, max_retries: int = 3) -> str:
        """Call LLM with retry"""
        for attempt in range(max_retries):
            try:
                messages = [{"role": "user", "content": prompt}]
                response = self.llm_client.create_completion(messages=messages, stream=True)
                result = self.llm_client.handle_streaming_response(response)
                return result.get("content", "") if isinstance(result, dict) else str(result)
            except Exception as e:
                self.console.print(f"⚠️ LLM call failed (attempt {attempt+1}): {str(e)}")
                time.sleep(1)
        return ""

    def _clean_json(self, raw_response: str) -> str:
        """Clean raw response to valid JSON"""
        response = raw_response.strip()
        if "```json" in response:
            start = response.find("```json") + 7
            end = response.rfind("```")
            if start > 6 and end > start:
                return response[start:end].strip()
        start = max(response.find("{"), response.find("["))
        end = max(response.rfind("}") + 1, response.rfind("]") + 1)
        return response[start:end] if start != -1 and end != 0 else "[]"

class TaskAnalyzer(Agent):
    """Task analyzer and decomposer for software engineering tasks"""

    AVAILABLE_TOOLS = [
        "fs_read", "fs_write", "execute_bash", "code_interpreter", "introspect", "debate_agent"
    ]

    def __init__(self, config: AgentConfig):
        super().__init__(config)
        self.console = Console()
        self.json_repair = JsonSchemaRepairAgent(config)

    def analyze(self, user_input: str) -> AnalysisResult:
        """Analyze input and create task plan"""
        try:
            self.console.print("📋 Decomposing into tasks...")
            tasks = self._decompose_tasks(user_input)
            self.console.print(f"✅ Created {len(tasks)} tasks")

            self.console.print("🔄 Building execution plan...")
            execution_plan = self._build_execution_plan(tasks)
            self.console.print(f"✅ Execution plan ready with {execution_plan.total_actions} actions")

            tools_needed = list(set(tool for task in tasks for tool in task.tools))
            success_criteria = self._define_success_criteria(user_input, tasks)
            overall_resources = self._aggregate_resources(tasks)

            self.console.print("✅ Analysis complete!")
            return AnalysisResult(
                tasks=tasks,
                execution_plan=execution_plan,
                tools_needed=tools_needed,
                success_criteria=success_criteria,
                resources=overall_resources
            )

        except Exception as e:
            self.console.print(f"❌ Analysis error: {str(e)}")
            return self._create_fallback_result(user_input)

    def _decompose_tasks(self, user_input: str) -> List[Task]:
        """Decompose input into tasks with repair"""
        prompt = f"""
Decompose this software engineering task into a sequence of technical tasks: '{user_input}'

Available tools: {', '.join(self.AVAILABLE_TOOLS)}

Rules:
- First task MUST be 'Requirement Analysis' using tools like code_interpreter or introspect.
- If user input omits frameworks/libraries, propose potential ones in Requirement Analysis, then use debate_agent to finalize selections.
- Focus on technical decomposition (e.g., code, testing, deployment). No human-like actions.
- Each action step must have self-sufficient system_prompt and user_prompt.
- Invoke introspect after each subtask with a structured introspect_prompt.
- Identify dependencies/resources, sequence/parallel for tasks/subtasks.
- Resources: Track previous and required resources.

Generate a list of tasks in JSON format. Each task:
- id: string (e.g., T001)
- name: string
- description: string
- dependencies: list of strings
- tools: list of strings (from available tools)
- actions: list of objects with step (int), action (str), tool (str or null), purpose (str), sub_steps (list str), introspect_after (bool), system_prompt (str), user_prompt (str), introspect_prompt (str), resources (object with previous/required lists), execution_mode ("sequential" or "parallel")
- success_criteria: string
- expected_outputs: list of strings
- resources: object with previous/required lists

Output as JSON array: [...]
"""
        response = self._call_llm_with_retry(prompt)
        self.console.print(f"[DEBUG] Raw LLM response: {response}")

        task_data = self.json_repair.repair_json(response, user_input)
        tasks = []
        for idx, data in enumerate(task_data):
            try:
                tasks.append(Task(**data))
            except ValidationError as ve:
                self.console.print(f"⚠️ Task {idx+1} invalid: {ve}")
                continue
        return tasks if tasks else self._create_fallback_tasks(user_input)

    def _build_execution_plan(self, tasks: List[Task]) -> ExecutionPlan:
        """Build execution plan from tasks"""
        task_dict = {task.id: task for task in tasks}
        dep_graph = {task.id: set(task.dependencies) for task in tasks}
        
        phases = []
        visited = set()
        independents = [tid for tid, deps in dep_graph.items() if not deps]
        
        while independents:
            phase = sorted(independents)
            phases.append(phase)
            visited.update(phase)
            new_independents = []
            for tid in list(dep_graph.keys()):
                if tid not in visited:
                    dep_graph[tid] -= set(phase)
                    if not dep_graph[tid]:
                        new_independents.append(tid)
            independents = new_independents
        
        parallel_groups = [phase for phase in phases if len(phase) > 1]
        critical_path = [tid for phase in phases for tid in phase[:1]]
        total_actions = sum(len(task.actions) for task in tasks)
        ordering = [tid for phase in phases for tid in phase]
        
        return ExecutionPlan(
            sequential_phases=phases,
            parallel_groups=parallel_groups,
            critical_path=critical_path,
            total_actions=total_actions,
            ordering=ordering
        )

    def _define_success_criteria(self, user_input: str, tasks: List[Task]) -> List[str]:
        """Define overall success criteria"""
        criteria = ["All technical subtasks completed with positive introspect feedback"]
        unique_criteria = set(task.success_criteria for task in tasks)
        criteria.extend(list(unique_criteria)[:5])
        return criteria

    def _aggregate_resources(self, tasks: List[Task]) -> Resources:
        """Aggregate overall resources"""
        previous = set()
        required = set()
        for task in tasks:
            previous.update(task.resources.previous)
            required.update(task.resources.required)
            for action in task.actions:
                previous.update(action.resources.previous)
                required.update(action.resources.required)
        return Resources(previous=list(previous), required=list(required))

    def _call_llm_with_retry(self, prompt: str, max_retries: int = 3) -> str:
        """Call LLM with retry"""
        for attempt in range(max_retries):
            try:
                messages = [{"role": "user", "content": prompt}]
                response = self.llm_client.create_completion(messages=messages, stream=True)
                result = self.llm_client.handle_streaming_response(response)
                return result.get("content", "") if isinstance(result, dict) else str(result)
            except Exception as e:
                self.console.print(f"⚠️ LLM call failed (attempt {attempt+1}): {str(e)}")
                time.sleep(1)
        return ""

    def _create_fallback_tasks(self, user_input: str) -> List[Task]:
        """Fallback tasks if decomposition fails"""
        return [Task(
            id="T001",
            name="Requirement Analysis and Execution",
            description=f"Analyze and execute: {user_input}",
            dependencies=[],
            tools=["introspect", "code_interpreter"],
            actions=[
                ActionStep(
                    step=1,
                    action="Analyze requirements",
                    tool="introspect",
                    purpose="Parse technical needs",
                    sub_steps=["Extract specs"],
                    introspect_after=True,
                    system_prompt="You are a technical agent for software tasks.",
                    user_prompt=f"Analyze technical aspects of: {user_input}. Propose frameworks if unspecified.",
                    introspect_prompt="Evaluate {performed_action} against task '{user_input}'. Criteria: correctness, completeness, alignment.",
                    resources=Resources(previous=[], required=["user_input"]),
                    execution_mode=ExecutionMode.SEQUENTIAL
                ),
                ActionStep(
                    step=2,
                    action="Implement solution",
                    tool="code_interpreter",
                    purpose="Execute code",
                    sub_steps=["Write and run"],
                    introspect_after=True,
                    system_prompt="You are a technical agent for software tasks.",
                    user_prompt=f"Implement based on analysis of: {user_input}",
                    introspect_prompt="Evaluate {performed_action} against task '{user_input}'. Criteria: correctness, completeness, alignment.",
                    resources=Resources(previous=["analysis_output"], required=["code_environment"]),
                    execution_mode=ExecutionMode.SEQUENTIAL
                )
            ],
            success_criteria="Technical requirements met with positive feedback",
            expected_outputs=["implementation"],
            resources=Resources(previous=[], required=["user_input"])
        )]

    def _create_fallback_result(self, user_input: str) -> AnalysisResult:
        """Fallback result if analysis fails"""
        tasks = self._create_fallback_tasks(user_input)
        execution_plan = self._build_execution_plan(tasks)
        tools_needed = list(set(tool for task in tasks for tool in task.tools))
        success_criteria = self._define_success_criteria(user_input, tasks)
        overall_resources = self._aggregate_resources(tasks)
        return AnalysisResult(
            tasks=tasks,
            execution_plan=execution_plan,
            tools_needed=tools_needed,
            success_criteria=success_criteria,
            resources=overall_resources
        )

    def analyze_to_json(self, user_input: str) -> Dict[str, Any]:
        """Analyze and return JSON dict"""
        result = self.analyze(user_input)
        return result.model_dump(mode='json')


In [13]:
# | export
config = AgentConfig(name="task_analyzer", model="gpt-oss:20b")
analyzer = TaskAnalyzer(config)
result = analyzer.analyze_to_json(
    "Design and implement a generic, configuration-driven time series forecasting framework that supports multiple models, automated preprocessing, feature engineering, hyperparameter tuning, and scalable deployment"
)
print(json.dumps(result, indent=2))


2025-09-25 17:11:51,481 - INFO - HTTP Request: POST http://192.168.29.147:11500/v1/chat/completions "HTTP/1.1 200 OK"
2025-09-25 17:11:51,537 - INFO - Initialized LLM client with model: gpt-oss:20b
2025-09-25 17:11:51,782 - INFO - HTTP Request: POST http://192.168.29.147:11500/v1/chat/completions "HTTP/1.1 200 OK"
2025-09-25 17:11:51,835 - INFO - Initialized LLM client with model: gpt-oss:20b


📋 Decomposing into tasks...

2025-09-25 17:11:52,648 - INFO - HTTP Request: POST http://192.168.29.147:11500/v1/chat/completions "HTTP/1.1 200 OK"


🏃 View run marvelous-donkey-995 at: http://localhost:5000/#/experiments/809751183417148104/runs/f63d9106f69c4bdebcdcc477d5c96f2f
🧪 View experiment at: http://localhost:5000/#/experiments/809751183417148104

╭─────────────────────── 🤔 Thinking ───────────────────────╮
│ We need to output tasks decomposition in JSON format. Must start with Requirement Analysis. Each task includes actions. Need introspect after each subtask. For introspecting, provide introspect_prompt. Tools: introspect (likely for introspection). Also other tools: fs_read, fs_write, execute_bash, code_interpreter, debate_agent.

We need to provide tasks covering design and implementation of generic forecasting framework. Outline tasks like: Requirement Analysis, Selection of libraries/frameworks, Architecture Design, Data ingestion module, Preprocessing pipeline, Feature engineering module, Model integration (multiple models), Hyperparameter tuning component, Validation and evaluation, Deployment pipeline, Monitoring an

[

...

🔄 Building execution plan...

✅ Execution plan ready with 2 actions

✅ Analysis complete!

{
  "tasks": [
    {
      "id": "T001",
      "name": "Requirement Analysis and Execution",
      "description": "Analyze and execute: Design and implement a generic, configuration-driven time series forecasting framework that supports multiple models, automated preprocessing, feature engineering, hyperparameter tuning, and scalable deployment",
      "dependencies": [],
      "tools": [
        "introspect",
        "code_interpreter"
      ],
      "actions": [
        {
          "step": 1,
          "action": "Analyze requirements",
          "tool": "introspect",
          "purpose": "Parse technical needs",
          "sub_steps": [
            "Extract specs"
          ],
          "introspect_after": true,
          "system_prompt": "You are a technical agent for software tasks.",
          "user_prompt": "Analyze technical aspects of: Design and implement a generic, configuration-driven time series forecasting framework that supports multiple models, automated preprocessing, f

[Trace(trace_id=tr-563494e4cfcae639043eb84dc2a22b39), Trace(trace_id=tr-e3b321856f40dc015a8e58943f8e50c9), Trace(trace_id=tr-2df2664576c7b95be0c63d3c6901a10c), Trace(trace_id=tr-a8ff308dd2f772e694871a7750076cea)]

In [14]:
result['tasks']

[{'id': 'T001',
  'name': 'Requirement Analysis and Execution',
  'description': 'Analyze and execute: Design and implement a generic, configuration-driven time series forecasting framework that supports multiple models, automated preprocessing, feature engineering, hyperparameter tuning, and scalable deployment',
  'dependencies': [],
  'tools': ['introspect', 'code_interpreter'],
  'actions': [{'step': 1,
    'action': 'Analyze requirements',
    'tool': 'introspect',
    'purpose': 'Parse technical needs',
    'sub_steps': ['Extract specs'],
    'introspect_after': True,
    'system_prompt': 'You are a technical agent for software tasks.',
    'user_prompt': 'Analyze technical aspects of: Design and implement a generic, configuration-driven time series forecasting framework that supports multiple models, automated preprocessing, feature engineering, hyperparameter tuning, and scalable deployment. Propose frameworks if unspecified.',
    'introspect_prompt': "Evaluate {performed_acti